In [1]:
import cv2
import os
from pathlib import Path
from tqdm import tqdm

In [6]:

# Function to apply CLAHE to an image (handles both colored and grayscale)
def apply_clahe(image):
    if len(image.shape) == 3:  # Colored image (3 channels)
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l_channel = lab[:, :, 0]
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        clahe_img = clahe.apply(l_channel)
        lab[:, :, 0] = clahe_img
        result_image = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    else:  # Grayscale image (1 channel)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        result_image = clahe.apply(image)
    
    return result_image

In [7]:
# Path to the folder containing your original images
input_folder = "C:/Users/SANYA CHANDEL/Desktop/combined_images"

# Path to the folder where you want to save the processed images
output_folder = "C:/Users/SANYA CHANDEL/Desktop/clahe_output_dataset"


In [8]:
# Create the output folder if it doesn't exist
Path(output_folder).mkdir(parents=True, exist_ok=True)

In [9]:
# Loop through each class folder in the input folder
for class_folder in os.listdir(input_folder):
    class_path = os.path.join(input_folder, class_folder)
    
    # Create a corresponding class folder in the output folder
    output_class_path = os.path.join(output_folder, class_folder)
    Path(output_class_path).mkdir(parents=True, exist_ok=True)
    
    # Loop through each image in the class folder
    for image_file in tqdm(os.listdir(class_path)):
        image_path = os.path.join(class_path, image_file)
        
        # Read the image
        img = cv2.imread(image_path)
        
        # Check if the image is not None before processing
        if img is not None:
            # Handle colored and grayscale images
            if len(img.shape) == 3:  # Colored image (3 channels)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            elif len(img.shape) == 2:  # Grayscale image (1 channel)
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            
            # Apply CLAHE
            processed_img = apply_clahe(img)
            
            # Save the processed image to the output folder
            output_path = os.path.join(output_class_path, image_file)
            cv2.imwrite(output_path, cv2.cvtColor(processed_img, cv2.COLOR_RGB2BGR))
        else:
            print(f"Failed to read image: {image_path}")

100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:02<00:00, 126.61it/s]


Failed to read image: C:/Users/SANYA CHANDEL/Desktop/combined_images\broadleaf\desktop.ini


100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:03<00:00, 86.22it/s]


In [14]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path to the processed dataset
dataset_folder = "C:/Users/SANYA CHANDEL/Desktop/clahe_output_dataset"

# Path to the folder where you want to save the train and test sets
output_train_folder = "C:/Users/SANYA CHANDEL/Desktop/clahe_output_train"
output_test_folder = "C:/Users/SANYA CHANDEL/Desktop/clahe_output_test"

# Create train and test folders if they don't exist
Path(output_train_folder).mkdir(parents=True, exist_ok=True)
Path(output_test_folder).mkdir(parents=True, exist_ok=True)

# Loop through each class folder in the processed dataset
for class_folder in os.listdir(dataset_folder):
    class_path = os.path.join(dataset_folder, class_folder)
    
    # Create corresponding class folders in train and test sets
    train_class_path = os.path.join(output_train_folder, class_folder)
    test_class_path = os.path.join(output_test_folder, class_folder)
    Path(train_class_path).mkdir(parents=True, exist_ok=True)
    Path(test_class_path).mkdir(parents=True, exist_ok=True)
    
    # List all images in the class folder
    images = [img for img in os.listdir(class_path) if img.endswith(".jpg")]
    
    # Split the images into train and test sets (80:20 ratio)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Copy images to train folder
    for train_image in train_images:
        src_path = os.path.join(class_path, train_image)
        dest_path = os.path.join(train_class_path, train_image)
        shutil.copy(src_path, dest_path)
    
    # Copy images to test folder
    for test_image in test_images:
        src_path = os.path.join(class_path, test_image)
        dest_path = os.path.join(test_class_path, test_image)
        shutil.copy(src_path, dest_path)
